In [1]:
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import os


In [2]:
def histogram_intersection(p, q):
    # Calculate histogram intersection
    # q is the reference distribution
    # p is the query distribution
    
    minima = np.minimum(p, q)
    hi = np.true_divide(np.sum(minima), np.sum(p))
    
    if hi > 1 or hi < 0:
        print('Error, HI =', hi)
        print(p)
        print(q)
        return
    return hi


def chi_square_distance(p, q):
    # Calculate chi-square distance
    # q is the reference distribution
    # p is the query distribution
    
    p = np.array(p)/np.sum(p)
    q = np.array(q)/np.sum(q)
    
    p = np.array(p) + 1
    q = np.array(q) + 1
    
    return np.sum(((p - q)**2 / (p + q))) / 2


def kl_divergence(p, q):
    # Calculate Kullback-Leibler Divergence
    # q is the reference distribution
    # p is the query distribution
    
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)
    
    p = np.array(p) + 1
    q = np.array(q) + 1
    
    p /= p.sum()
    q /= q.sum()
    
    kl_div = np.sum(p * np.log(p / q))
    return kl_div


def earth_movers_distance(p, q):
    # Calculate Earth Mover's Distance
    # q is the reference distribution
    # p is the query distribution
    
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)

    # Normalize the distributions to ensure they sum to 1
    p /= p.sum()
    q /= q.sum()

    # Calculate cumulative distributions
    P = np.cumsum(p)
    Q = np.cumsum(q)

    # Calculate the cost matrix
    C = np.abs(np.subtract.outer(P, Q))

    # Solve linear sum assignment problem
    row_ind, col_ind = linear_sum_assignment(C)
    emd = C[row_ind, col_ind].sum()

    return emd


def kolmogorov_smirnov_distance(p, q):
    
    # Calculate KS distance
    # q is the reference distribution
    # p is the query distribution
    
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)

    # Normalize the distributions to ensure they sum to 1
    p /= p.sum()
    q /= q.sum()

    # Calculate cumulative distributions
    P = np.cumsum(p)
    Q = np.cumsum(q)

    ks_distance = np.max(np.abs(P - Q))

    return ks_distance


def rank_probability_score(p, q):
    
    # Calculate the ranked probability score
    # q is the reference distribution
    # p is the query distribution
    
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)

    # Normalize the distributions to ensure they sum to 1
    p /= p.sum()
    q /= q.sum()

    # Calculate cumulative distributions
    P = np.cumsum(p)
    Q = np.cumsum(q)

    rps = np.sum((P - Q)**2)

    return rps


def RDS(p, q):
    
    # Calculate Relative distribution shift
    # q is the reference distribution
    # p is the query distribution
    
    p_bins = len(p)
    p_obs = sum(p)
    
    q_bins = len(q)
    q_obs = sum(q)
    
    z_p = (p_bins + 1)/p_bins
    p = [sum(p[:ii+1])**(z_p) for ii in range(len(p))]
    Sp = np.sum(np.array(p)/(p_obs**z_p)) - 1
    Sp = Sp/(p_bins - 1)
    
    z_q = (q_bins + 1)/q_bins
    q = [sum(q[:ii+1])**(z_q) for ii in range(len(q))]
    Sq = np.sum(np.array(q)/(q_obs**z_q)) - 1
    Sq = Sq/(q_bins - 1)
    
    return Sq - Sp


In [3]:
def flatten_histogram(image, num_bins, htype='color'):
    
    if htype == 'color':
        """
        Construct a flattened 1D histogram for a color image by concatenating the histograms
        of each color channel.
        """
        if len(image.shape) == 3:  # Check if image is RGB
            channels = cv2.split(image)
            histograms = []
            for chan in channels:
                hist, _ = np.histogram(chan, bins=num_bins, range=(0, num_bins))
                histograms.append(hist)
            histogram = np.concatenate(histograms) / np.sum(histograms)
            
        else:  # For grayscale images, compute histogram directly
            histogram, _ = np.histogram(image, bins=num_bins, range=(0, num_bins))
            histogram = histogram / np.sum(histogram)
        return histogram
    
    elif htype == 'hue':
        """
        Construct a 1D hue histogram for a color image by converting it to the HSV color space
        and calculating the histogram of the hue component.
        """
        # Convert RGB image to HSV
        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        # Extract Hue component
        hue_channel = hsv_image[:, :, 0]

        # Calculate histogram of the Hue component
        histogram, _ = np.histogram(hue_channel, bins=num_bins, range=(0, 180))  # Hue values range from 0 to 180 in OpenCV

        # Normalize histogram
        histogram = histogram / np.sum(histogram)
    
        return histogram
    
    
    elif htype == 'intensity':
        """
        Construct a 1D intensity histogram for an image by converting it to grayscale
        and calculating the histogram based on the intensity values.
        """
        # Check if image is in grayscale
        if len(image.shape) == 3:  # If not, convert image to grayscale
            grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            grayscale_image = image  # The image is already in grayscale

        # Calculate the histogram of intensity values
        histogram, _ = np.histogram(grayscale_image, bins=num_bins, range=(0, 256))

        # Normalize
        histogram = histogram / np.sum(histogram)
        return histogram
    



def process_video_with_metric(video_path, metric_name, num_bins):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    metric_values = []
    frame_indices = []
    first_frame = None
    max_rds_value = None
    max_rds_frame = None

    # Define videos that need trimming
    videos_to_trim = ['data/time_lapse_video/855852-sd_640_360_30fps.mp4', 
                      'data/time_lapse_video/856030-sd_640_360_25fps.mp4',
                      'data/time_lapse_video/856006-sd_640_360_25fps.mp4',
                      'data/time_lapse_video/856990-sd_640_360_30fps.mp4',
                      'data/time_lapse_video/2038351-sd_640_360_30fps.mp4',
                      'data/time_lapse_video/855215-sd_640_360_24fps.mp4',
                      'data/time_lapse_video/3541930-sd_426_240_24fps.mp4',
                      'data/time_lapse_video/9255202-sd_426_240_25fps.mp4',
                      'data/time_lapse_video/9255065-sd_426_240_25fps.mp4',
                      'data/time_lapse_video/10881636-sd_640_360_25fps.mp4',
                      'data/time_lapse_video/10881635-sd_640_360_25fps.mp4',
                      'data/time_lapse_video/10881640-sd_640_360_25fps.mp4',
                      'data/time_lapse_video/14668-258508783_tiny.mp4',
                      'data/time_lapse_video/11440-230272102_tiny.mp4',
                      'data/time_lapse_video/9289-218679635_tiny.mp4',
                      'data/time_lapse_video/16986-277920487_tiny.mp4',
                      'data/time_lapse_video/8560-211218186_tiny.mp4',
                      'data/time_lapse_video/18020-287831424_tiny.mp4',
                      'data/time_lapse_video/81089-574743647_medium.mp4',
                      
                 ]

    # Determine if current video needs trimming
    trim = video_path in videos_to_trim

    # Adjust start and end indices if trimming is needed
    start_index = 10 if trim else 0
    end_index = total_frames - 10 if trim else total_frames

    for frame_count in range(total_frames):
        success, frame = video.read()
        if not success:
            break

        # Skip processing for trimmed frames
        if frame_count < start_index or frame_count >= end_index:
            continue

        if metric_name == 'RDS' and frame_count == start_index:
            first_frame = frame  # Adjusted to capture the first untrimmed frame

        if frame_count == start_index:  # Adjusted to use the first untrimmed frame as reference
            reference_hist = flatten_histogram(frame, num_bins, 'color')

        frame_hist = flatten_histogram(frame, num_bins, 'color')

        if metric_name == '1 - HI':
            metric_value = 1 - histogram_intersection(frame_hist, reference_hist)
        elif metric_name == 'Chi-Square':
            metric_value = chi_square_distance(frame_hist, reference_hist)
        elif metric_name == 'KL':
            metric_value = kl_divergence(frame_hist, reference_hist)
        elif metric_name == 'EMD':
            metric_value = earth_movers_distance(frame_hist, reference_hist)
        elif metric_name == 'KS':
            metric_value = kolmogorov_smirnov_distance(frame_hist, reference_hist)
        elif metric_name == 'RPS':
            metric_value = rank_probability_score(frame_hist, reference_hist)
        elif metric_name == 'RDS':
            metric_value = RDS(frame_hist, reference_hist)

        metric_values.append(metric_value)

        # Update max RDS frame and value if this is the RDS metric
        if metric_name == 'RDS' and (max_rds_value is None or abs(metric_value) > abs(max_rds_value)):
            max_rds_value = metric_value
            max_rds_frame = frame

        frame_indices.append(frame_count)

    video.release()
    return metric_values, frame_indices, first_frame, max_rds_frame, max_rds_value



def make_fig(inputs, lab, first_frame, max_rds_frame, max_rds_value):
    fs = 18
    fig = plt.figure(figsize=(18, 18))  # Adjusted size for 3x3 grid
    
    # Function to rotate an image if width > height
    def rotate_if_needed(image):
        if image.shape[1] > image.shape[0]:  # Check if width > height
            return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)  # Rotate 90 degrees clockwise
        return image

    # Apply rotation if needed and plot the first frame
    rotated_first_frame = rotate_if_needed(first_frame)
    ax1 = plt.subplot(3, 3, 1)
    ax1.imshow(cv2.cvtColor(rotated_first_frame, cv2.COLOR_BGR2RGB))
    ax1.set_title('First frame', fontsize=fs-2)
    ax1.axis('off')

    # Apply rotation if needed and plot the frame with the greatest absolute RDS value
    rotated_max_rds_frame = rotate_if_needed(max_rds_frame)
    ax2 = plt.subplot(3, 3, 2)
    ax2.imshow(cv2.cvtColor(rotated_max_rds_frame, cv2.COLOR_BGR2RGB))
    ax2.set_title(f'RDS: {max_rds_value:.3f}', fontsize=fs-2)
    ax2.axis('off')

    # Plot the existing metric plots in the rest of the grid
    for i, ls in enumerate(inputs, start=2):
        ax = plt.subplot(3, 3, i+1)
        plt.plot(ls[1], ls[0], c='k')
        plt.xlabel('Frame', fontsize=fs)
        plt.ylabel(ls[2], fontsize=fs, fontweight='bold')
        plt.tick_params(axis='both', labelsize=10)

    fig.patch.set_facecolor('white')
    plt.subplots_adjust(hspace=0.35, wspace=0.4)
    plt.savefig('Final_Figs/from_image_analysis/'+lab+'.pdf', bbox_inches='tight', format='pdf', dpi=600)
    plt.close()



In [5]:
# From pexels
pexel_lists = [['3541930-sd_426_240_24fps'],
         ['2559431-sd_426_240_24fps'],
         ['1110140-sd_640_360_30fps'],
         ['3053992-sd_426_240_30fps'],
         ['988047-sd_640_360_30fps'],
         ['5595352-sd_240_426_24fps'],
         ['856136-sd_640_360_25fps'],
         ['857251-sd_540_360_25fps'],
         ['2675512-sd_426_240_24fps'],
         ['855456-sd_640_360_30fps'],
         ['855210-sd_640_360_30fps'],
         ['2038351-sd_640_360_30fps'],
         ['855215-sd_640_360_24fps'],
         ['855852-sd_640_360_30fps'],
         ['856990-sd_640_360_30fps'],
         ['1779441-sd_640_360_30fps'],
         ['856006-sd_640_360_25fps'],
         ['856030-sd_640_360_25fps'],
         ['9837842-sd_960_540_30fps'],
         ['10661569-sd_426_240_30fps'],
         ['2731905-hd_1280_720_24fps'],
         ['4782177-hd_1280_720_30fps'],
         ['5158727-hd_1280_720_30fps'],
         ['14219843-sd_426_240_24fps'],
         ['14219844-sd_426_240_24fps'],
         ['3947230-sd_426_240_25fps'],
         ['10881640-sd_640_360_25fps'],
         ['10881635-sd_640_360_25fps'],
         ['10881636-sd_640_360_25fps'],
         ['4990234-sd_426_240_30fps'],
         ['6923433-sd_426_240_30fps'],
         ['4990232-sd_426_240_30fps'],
         ['4990233-sd_426_240_30fps'],
         ['9255202-sd_426_240_25fps'],
         ['7235032-sd_240_426_30fps'],
         ['7235041-sd_240_426_30fps'],
         ['5388577-sd_240_426_25fps'],
         ['5094592-sd_226_426_25fps'],
         ['7101966-hd_1080_1644_25fps'],
         ['17333461-sd_426_240_25fps'],
         ['4943067-sd_426_240_24fps'],
         ['7710243-sd_426_240_30fps'],
         ['10613972-hd_1920_1080_24fps'],
         ['10413970-hd_1920_1080_30fps'],
         ['7235033-sd_240_426_30fps'],
         ['10469592-sd_640_360_30fps'],
         ['4584697-hd_1920_1080_25fps'],
         ['7703989-hd_1080_1920_25fps'],
         ['7234900-hd_1920_1080_30fps'],
         ['5094592-hd_1080_2048_25fps'],
         ['7565623-hd_1080_1920_25fps'],
         ['5926164-hd_1920_1080_30fps'],
         ['8516544-hd_1080_1920_30fps'],
         ['4378289-hd_1920_1080_24fps'],
         ['1681023-sd_640_338_24fps'],
        ]


# From Pixabay
pixabay_lists = [['81092-574743664_medium'],
         ['81089-574743647_medium'],
         ['12902-242487547_small'],
         ['1378-147055474_medium'],
         ['21161-315224572_medium'],
         ['72512-543388307_medium'],
         ['8782-214159393_tiny'],
         ['7684-202716554_medium'],
         ['91592-628463727_medium'],
         ['18327-291012897_small'],
         ['188766-883827686_small'],
         ['55303-499594258_small'],
         ['90559-626984434_small'],
         ['12719-241674195_medium'],
         ['14124-254487707_tiny'],
         ['55554-502339972_small'],
         ['18324-291012885_small'],
         ['80645-572367655_medium'],
         ['18020-287831424_tiny'],
         ['75475-556034323_small'],
         ['24718-345209742_small'],
         ['37311-413624981_small'],
         ['8560-211218186_tiny'],
         ['16986-277920487_tiny'],
         ['151406-800680974_tiny'],
         ['144761-785265038_tiny'],
         ['9289-218679635_tiny'],
         ['11440-230272102_tiny'],
         ['14668-258508783_tiny'],
         ['12668-241236737_tiny'],
         ['122211-724743609_tiny'],
         ['122336-725067675_tiny'],
         ['24715-345209726_small'],
         ['66262-516980798_tiny'],
         ['4935-181466751_medium'],
         ['43781-436252322_tiny'],
         ['18251-290360026_tiny'],
         ['55776-503981063_small'],
         ['18435-292228551_tiny'],
         ['18433-292228530_tiny'],
         ['81185-575775322_tiny'],
         ['1285-145117054_small'],
         ['6089-188582377_tiny'],
         ['15708-266043570_medium'],
         ['18902-297379468_tiny'],
         ['22352-326722819_medium'],
         ['25694-352026464_small'],
         ['40487-425442645_small'],
         ['43448-436041766_tiny'],
         ['122213-724743622_tiny'],
         ['124277-730508581_medium'],
         ['131398-750568980_tiny'],
         ['135422-761676527_small'],
         ['135578-762107314_medium'],
         ['166284-834580727_tiny'],
         ['11440-230272102_tiny'],
         ['14668-258508783_tiny'],
         ['9289-218679635_tiny'],
         ['16986-277920487_tiny'],
         ['8560-211218186_tiny'],
         ['18020-287831424_tiny'],
         ['81089-574743647_medium'],
        ]

lists = pexel_lists + pixabay_lists

for ls in lists:
    lab = ls[0]
    video_path = 'data/time_lapse_video/'+lab+'.mp4'
    
    # Check if file exists
    if os.path.exists('Final_Figs/from_image_analysis/'+lab+'.pdf'):
        print('Already exists:', lab)
        continue
        
    else:
        print(ls)

    
    num_bins = 256
    EMD1, EMD1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'EMD', num_bins)
    HI1, HI1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, '1 - HI', num_bins)
    CS1, CS1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'Chi-Square', num_bins)
    KL1, KL1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'KL', num_bins)
    KS1, KS1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'KS', num_bins)
    RPS1, RPS1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'RPS', num_bins)
    RDS1, RDS1_frames, first_frame, max_rds_frame, max_rds_value = process_video_with_metric(video_path, 'RDS', num_bins)

    inputs = [[RDS1, RDS1_frames, 'RDS'],
              [EMD1, EMD1_frames, 'EMD'],
              [HI1, HI1_frames, '1 - HI'],
              [CS1, CS1_frames, 'CS'],
              [KL1, KL1_frames, 'KL'],
              [KS1, KS1_frames, 'KS'],
              [RPS1, RPS1_frames, 'RPS']]
    make_fig(inputs, lab, first_frame, max_rds_frame, max_rds_value)



Already exists: 3541930-sd_426_240_24fps
Already exists: 2559431-sd_426_240_24fps
Already exists: 1110140-sd_640_360_30fps
Already exists: 3053992-sd_426_240_30fps
Already exists: 988047-sd_640_360_30fps
Already exists: 5595352-sd_240_426_24fps
Already exists: 856136-sd_640_360_25fps
Already exists: 857251-sd_540_360_25fps
Already exists: 2675512-sd_426_240_24fps
Already exists: 855456-sd_640_360_30fps
Already exists: 855210-sd_640_360_30fps
Already exists: 2038351-sd_640_360_30fps
Already exists: 855215-sd_640_360_24fps
Already exists: 855852-sd_640_360_30fps
Already exists: 856990-sd_640_360_30fps
Already exists: 1779441-sd_640_360_30fps
Already exists: 856006-sd_640_360_25fps
Already exists: 856030-sd_640_360_25fps
Already exists: 9837842-sd_960_540_30fps
Already exists: 10661569-sd_426_240_30fps
Already exists: 2731905-hd_1280_720_24fps
Already exists: 4782177-hd_1280_720_30fps
Already exists: 5158727-hd_1280_720_30fps
Already exists: 14219843-sd_426_240_24fps
Already exists: 14219